In [14]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
import dash_bootstrap_components as dbc
import dizionari
import datetime #TODO: eliminarla se non utilizzata
import numpy as np


data = pd.read_csv("../data/ISTAT_Italian_Excursion_2014_2022.csv")

# CREO UN NUOVO DATAFRAME PIU' VERSATILE
# Nota: Nel nostro dataframe non sono presenti valori nulli [data.isnull().sum()]

# Rimozione di colonne ridondanti e superflui
colonne_da_eliminare = ["ID", "progind", "sesso", "staciv4", "condogg", "posiz4", "ateco3", "cond4", "ORARIO", "RAPP", "CODPROF7", "progesc", "PARTESC", "NPARTESC"]
df = data.drop(columns=colonne_da_eliminare)

# Rinomina delle colonne
df.rename(columns = dizionari.dizionario_nomi, inplace=True)

# Creazione di una colonna 'Data' nel formato (ANNO-mese)
df['Data'] = pd.to_datetime(df['Anno'].apply(str) + '-' + df['Mese'].apply(str), format='%Y-%m')
df["Data"] = df["Data"].dt.strftime("%Y-%m")

# Colonna Età nel formato (anno)
df['Età'] = df['Età'].map(dizionari.eta_mapping)
df['Regione'] = (df['Regione']/10).astype(int)
#df['Nomi Destinazione (città)'] = df['Destinazione (città)'].map(dizionari.destinazione_citta_mapping)

conteggio_arrivi = df['Destinazione (città)'].value_counts().reset_index(name="Numero escursioni")

fig = px.choropleth(
            conteggio_arrivi,
            geojson='https://raw.githubusercontent.com/openpolis/geojson-italy/master/geojson/limits_IT_provinces.geojson',
            locations='Destinazione (città)',
            color='Numero escursioni',
            color_continuous_scale='Reds',
            featureidkey='properties.prov_istat_code_num',
            #animation_frame='Date',
            range_color=(0, max(conteggio_arrivi['Numero escursioni']))
            )

fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.update_geos(fitbounds="locations", visible=True)
fig.show()